Course Instructor: Bernd Neumayr, JKU

# UE04- SPARQL Updata and RDF Datasets

Complete the **8 tasks (1 point per task)** in the `3. SPARQL Update` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- Your solution in executable form: your solutions for SemAI.jar will make use of the default grap. In this notebook you have to transform your solutions according to the workaround exemplified in V04_SPARQL_Update.ipynb
- After executing the update request, print a serizalization of the dataset in TriG format.  

**Task 9 (2 points)**  is to develop a nice visualization of RDF datasets using `visualize_graph_pyvis` from UE02 as a starting point. The requirements are as follows:
- Each named graph must be represented as an independent graph. This means, for example, that :Jane in :JanesGraph is a different node than :Jane in :BillsGraph. There are no edges between nodes in different graphs.
- It is not strictly necessary to draw a box around each named graph, as seen in the slides. The different named graphs should simply be visually distinguishable and not overlap.
- If not all nodes within a named graph are connected, make sure in the visualization that the named graph still forms a coherent visual unit in some way.

In [48]:
# Install required packages
!pip install -q rdflib     # comment to avoid re-install with every re-run

# Imports
import pandas as pd
import rdflib
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace


# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as

# Task 1
Sie beginnen mit einem leeren Dataset. Fügen Sie in den Default Graph Statements ein, die sagen, dass :Peter der Autor von :G1 ist, und :Mary Autor von :G2. 



In [49]:
sol1 = """INSERT DATA {
  :G1 :author :Mary.
  :G2 :author :Peter.
}"""

ds = rdflib.Dataset()
ds.parse(format="trig", data="""
  @prefix :    <http://example.org/> .
  @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  .
  
  :main {
    :G1 :author :Mary .
    :G2 :author :Peter .
    }
  
  """)


print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .

:main {
    :G1 :author :Mary .

    :G2 :author :Peter .
}




# Task 2
Schreiben Sie { :Mary :knows :Peter, :John, :Mary. } in den Named Graph :G1 und { :Peter :knows :Mary. :John :knows :Mary. } in den Named Graph :G2. 

In [50]:
sol2 = """
INSERT DATA {
  GRAPH :G1 {
    :Mary :knows :Peter, :John, :Mary.
  }
  GRAPH :G2 {
    :Peter :knows :Mary.
    :John :knows :Mary.
  }
}"""

ds.parse(format="trig", data="""
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix :    <http://example.org/>

GRAPH :G1 {
    :Mary :knows :Mary , :John , :Peter .
  }
GRAPH :G2 {
    :Peter :knows :Mary . 
    :John :knows :Mary .
  }

""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}

:main {
    :G1 :author :Mary .

    :G2 :author :Peter .
}




# Task 3
Fragen Sie mittels INSERT-WHERE die :knows-Beziehungen aus :G2 ab und fügen deren inverse :knownBy-Beziehungen in den Default-Graph ein.
Ihr Update Request darf nicht enthalten: [Mary, Peter, John]

In [51]:
ds.update("""
INSERT {
  GRAPH :main {
    ?friend :knownBy ?person .
  }
}
WHERE {
  GRAPH :G2 {
    ?person :knows ?friend .
  }
}""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}

:main {
    :G1 :author :Mary .

    :G2 :author :Peter .

    :Mary :knownBy :John,
            :Peter .
}




# Task 4
Löschen Sie mittels DELETE-WHERE alle :knownBy-Beziehungen aus dem Default-Graph.
Ihr Update Request darf nicht enthalten: [Mary, Peter, John]

In [52]:
ds.update("""
DELETE {
  ?person :knownBy ?friend.
}
WHERE {
  ?person :knownBy ?friend.
}""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}

:main {
    :G1 :author :Mary .

    :G2 :author :Peter .

    :Mary :knownBy :John,
            :Peter .
}




# Task 5
Ermitteln Sie mittels INSERT-WHERE zu jedem Named-Graph dessen Anzahl an Statements mit der Property :knows und schreiben Sie diese in den Default Graph.
Ihr Update Request darf nicht enthalten: [G1, G2]

In [53]:
ds.update("""

INSERT {
  GRAPH :main {
    ?graph :knowsCount ?count
  }
}
WHERE {
  SELECT ?graph (COUNT(?count) AS ?count)
  WHERE {
    GRAPH ?graph {
      ?subject :knows ?object.
    }
  }
  GROUP BY ?graph
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}

:main {
    :G1 :author :Mary ;
        :knowsCount 0 .

    :G2 :author :Peter ;
        :knowsCount 0 .

    :Mary :knownBy :John,
            :Peter .
}


